## 1 : langchainを使ってみた。

参考記事URL：
- https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_retrieval
- https://qiita.com/harukikaneko/items/b004048f8d1eca44cba9

In [1]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


openai_api_key = os.environ.get("OPENAI_API_KEY")

loader = PyPDFLoader("https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf")
pages = loader.load_and_split()
print(pages[0])
chroma_index = Chroma.from_documents(pages, OpenAIEmbeddings())
docs = chroma_index.similarity_search("「フリーランスのリモートワークの実態」について教えて。", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

page_content='第１章
はじめに
2' metadata={'source': 'https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf', 'page': 1}


AttributeError: partially initialized module 'chromadb' has no attribute 'config' (most likely due to a circular import)

In [5]:
from langchain.text_splitter import CharacterTextSplitter
long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)
text_list = text_splitter.split_text(long_text)
print(text_list)
print(len(text_list))

document_list = text_splitter.create_documents([long_text])
print(document_list)
print(len(document_list))

368
['GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。\n\n自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。', 'これにより、人間と同じような文章を作成することが可能です。\n\nGPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。', 'GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。\n\nしかし、2021年9月までの情報しか持っていません。', 'このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。\n\nただし、GPT-4は完璧ではありません。\n\n時々、誤った情報や不適切な内容を生成することがあります。', '使用者は、その限界を理解し、\n\n適切な方法で利用することが重要です。']
5
[Document(page_content='GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。\n\n自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。'), Document(page_content='これにより、人間と同じような文章を作成することが可能です。\n\nGPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。'), Document(page_content='GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。\n\nしかし、2021年9月までの情報しか持っていません。'), Document(page_content='このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。\n\nただし、GPT-4は完璧ではありません。\n\n時々、誤った情報や不適切な内容を生成することがあります。'), Document(page_content='使用者は、その限界を理解し、\n\n適切な方法で利用することが重要です。')]
5


In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))
with open("./long_text.txt", "w") as f:
    f.write(long_text)
    f.close()

loader = TextLoader('./long_text.txt')

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
    text_splitter=text_splitter,
).from_loaders([loader])

query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)

query = "Q2. GPT4は第何世代のモデル？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)


368


/opt/anaconda3/envs/nnn_genai_webapp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


AttributeError: partially initialized module 'chromadb' has no attribute 'config' (most likely due to a circular import)